In [ ]:
# testing ksig  ---- clearly something wrong, since the values dont converge as order->infty for ksig

import time

def calc_iisig_kernel(X, Y, order):
    sig_X, sig_Y = streams_to_sigs([X,Y], order, disable_tqdm=True)
    dot = 1 + np.dot(sig_X, sig_Y)
    return dot


def calc_sigpde_kernel(X,Y):
    dyadic_order = 5
    static_kernel = sigkernel.LinearKernel()
    vv, uv = case_sig_pde([X], [Y], dyadic_order, static_kernel)
    return uv[0,0]


def calc_ksig_kernel(X,Y, order):
    import ksig
    static_kernel = ksig.static.kernels.LinearKernel() 
    sig_kernel = ksig.kernels.SignatureKernel(n_levels=order, order=1, static_kernel=static_kernel, normalize=False)
    dot = sig_kernel(np.array([X,X]), np.array([Y,Y]))[0,0]
    return dot


def trunc_sig_kernel(s1:np.ndarray, 
                    s2:np.ndarray, 
                    order:int, #order is truncation level of the signature
                    static_kernel_gram:Callable = linear_kernel_gram,
                    only_last:bool = True,

                    ):
    """s1 and s2 are time series of shape (T_i, d)"""
    K = static_kernel_gram(s1, s2)
    nabla = K[1:, 1:] + K[:-1, :-1] - K[1:, :-1] - K[:-1, 1:]
    sig_kers = jitted_trunc_sig_kernel(nabla, order)
    if only_last:
        return sig_kers[-1]
    else:
        return sig_kers



@njit
def reverse_cumsum(arr:np.ndarray, axis:int): #ndim=2
    """JITed reverse cumulative sum along the specified axis.
    (np.cumsum with axis is not natively supported by Numba)"""
    A = arr.copy()
    if axis==0:
        for i in np.arange(A.shape[0]-2, -1, -1):
            A[i, :] += A[i+1, :]
    else: #axis==1
        for i in np.arange(A.shape[1]-2, -1, -1):
            A[:,i] += A[:,i+1]
    return A


@njit
def jitted_trunc_sig_kernel(nabla:np.ndarray, # gram matrix (T_1, T_2)
                            order:int,
                            ):
    """Given difference matrix nabla_ij = K[i+1, j+1] + K[i, j] - K[i+1, j] - K[i, j+1],
    computes the truncated signature kernel of all orders up to 'order'."""
    B = np.ones((order+1, order+1, order+1, *nabla.shape))
    for d in np.arange(order):
        for n in np.arange(order-d):
            for m in np.arange(order-d):
                B[d+1,n,m] = 1 + nabla/(n+1)/(m+1)*B[d, n+1, m+1]
                r1 = reverse_cumsum(nabla * B[d, n+1, 1] / (n+1), axis=0)
                B[d+1,n,m, :-1, :] += r1[1:, :]
                r2 = reverse_cumsum(nabla * B[d, 1, m+1] / (m+1), axis=1)
                B[d+1,n,m, :, :-1] += r2[:, 1:]
                rr = reverse_cumsum(nabla * B[d, 1, 1], axis=0)
                rr = reverse_cumsum(rr, axis=1)
                B[d+1,n,m, :-1, :-1] += rr[1:, 1:]

    return B[:,0,0,0,0]
    
    



d = 2
MAX_ORDER = 18
times_iisig = np.zeros( (MAX_ORDER) )
times_sigker  = np.zeros( (MAX_ORDER) )
times_ksig = np.zeros( (MAX_ORDER) )
np.random.seed(99)
X, Y = np.random.randn(2, 19, d)/np.sqrt(d)
for order in range(1, MAX_ORDER+1):
    print("\norder", order)
    t0= time.time()
    dot1=calc_iisig_kernel(X, Y, order)
    t1 = time.time()
    dot2=trunc_sig_kernel(X, Y, order)
    t2 = time.time()
    dot3=calc_ksig_kernel(X, Y, order)
    t3 = time.time()
    times_iisig[order-1] = t1-t0
    times_sigker[order-1] = t2-t1
    times_ksig[order-1] = t3-t2
    print("dot1", dot1)
    print("dot2", dot2)
    print("dot3", dot3)

print("\n")
dot4 = calc_sigpde_kernel(X, Y)
print("dot_pde", dot4)


print("\ncomparison", times_iisig[1:]/times_sigker[1:])
print("\niisig", times_iisig[1:])
print("\nsigker", times_sigker[1:])
print("\nksig", times_ksig[1:])
print("\ncomparison", times_sigker[1:]/times_ksig[1:])